<a href="https://colab.research.google.com/github/ColdCoffee21/Foundations-of-Data-Science/blob/master/Outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Data Science/salaries.csv')
df.head(10)

,salary
0,96454
1,29557
2,47175
3,12419
4,33333333
5,90938
6,76404
7,85815
8,9999999
9,73611


In [ ]:
df.shape

(6000, 1)

In [ ]:
data_mean, data_std = np.mean(df['salary']), np.std(df['salary'])

In [ ]:
cut_off = data_std * 3

In [ ]:
lower, upper = data_mean-cut_off, data_mean+cut_off

In [ ]:
outliers = [x for x in df['salary'] if (x<lower) or (x>upper)]
outliers

[33333333,
 66666666,
 88888888,
 99999999,
 66666666,
 99999999,
 55555555,
 66666666,
 22222222,
 22222222,
 55555555,
 88888888,
 99999999,
 66666666,
 55555555,
 33333333,
 99999999,
 55555555,
 55555555,
 55555555,
 99999999,
 88888888,
 99999999,
 55555555,
 99999999]

In [ ]:
outliers_removed = [x for x in df['salary'] if (x>lower) or (x<upper)]
outliers_removed

[96454,
 29557,
 47175,
 12419,
 33333333,
 90938,
 76404,
 85815,
 9999999,
 73611,
 88624,
 61980,
 21875,
 83817,
 77572,
 57788,
 13968,
 85978,
 43388,
 80532,
 64804,
 15328,
 27068,
 32286,
 18678,
 36545,
 96768,
 76381,
 52003,
 61701,
 54058,
 60690,
 71779,
 45335,
 63292,
 84293,
 52007,
 38351,
 83067,
 5555555,
 99958,
 66120,
 33051,
 26564,
 52418,
 22375,
 25176,
 37110,
 63849,
 22018,
 18214,
 80841,
 40513,
 43311,
 98268,
 66666666,
 48007,
 35254,
 45312,
 24199,
 12480,
 12873,
 14793,
 74507,
 19384,
 53896,
 55719,
 7777777,
 66184,
 77321,
 39967,
 33767,
 27917,
 70816,
 30798,
 79174,
 17986,
 39009,
 14771,
 49949,
 15624,
 96491,
 60988,
 21283,
 92711,
 45217,
 39819,
 66941,
 88888888,
 98986,
 15199,
 42262,
 56765,
 23265,
 70128,
 94359,
 27628,
 42275,
 79769,
 93863,
 98629,
 70142,
 13971,
 73415,
 12951,
 51938,
 99999999,
 71644,
 79829,
 15370,
 65770,
 53309,
 84171,
 54610,
 31942,
 26105,
 47319,
 70199,
 21103,
 25486,
 99077,
 61626,
 30341

In [ ]:
data_mean

349050.10216666665

In [ ]:
data_std

4758049.22084576

In [ ]:
print("Number of outliers %d" % len(outliers))
print("Number of non outliers %d" % len(outliers_removed))

Number of outliers 25
Number of non outliers 6000


In [ ]:
print("Q2 quantile of arr : ", df.quantile(.50)) 
print("Q1 quantile of arr : ", df.quantile(.25)) 
print("Q3 quantile of arr : ", df.quantile(.75)) 
print("100th quantile of arr : ", df.quantile(.1))
print("iqr : ", df.quantile(.75) - df.quantile(.25))

Q2 quantile of arr :  salary    54663.0
Name: 0.5, dtype: float64
Q1 quantile of arr :  salary    32628.5
Name: 0.25, dtype: float64
Q3 quantile of arr :  salary    77410.0
Name: 0.75, dtype: float64
100th quantile of arr :  salary    18714.5
Name: 0.1, dtype: float64
iqr :  salary    44781.5
dtype: float64


In [ ]:
q1,q2,q3=df['salary'].quantile([.25,.5,.75])
iqr=q3-q1
out=[p for p in df['salary'] if (p<q1-1.5*iqr) or (p>q3+1.5*iqr)]
len(out)

30

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
model = IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1), max_features=1.0, random_state=1)
model.fit(df[['salary']])

IsolationForest(behaviour='deprecated', bootstrap=False, contamination=0.1,
                max_features=1.0, max_samples='auto', n_estimators=50,
                n_jobs=None, random_state=1, verbose=0, warm_start=False)

In [ ]:
df['scores'] = model.decision_function(df[['salary']])
df['anomaly'] = model.predict(df[['salary']])
df.head(10)

,salary,scores,anomaly
0,96454,0.002101,1
1,29557,0.042056,1
2,47175,0.090657,1
3,12419,-0.045839,-1
4,33333333,-0.234977,-1
5,90938,0.027426,1
6,76404,0.071841,1
7,85815,0.067033,1
8,9999999,-0.161412,-1
9,73611,0.070507,1


In [ ]:
an = df.loc[df['anomaly']==-1]
an_ind = list(an.index)
print(an.shape)

(600, 3)


In [ ]:
#how they got anomaly count

In [ ]:
#Local outlier factor
from sklearn.neighbors import LocalOutlierFactor
model = LocalOutlierFactor(novelty=True)
model.fit(df[['salary']])

LocalOutlierFactor(algorithm='auto', contamination='auto', leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=20, novelty=True, p=2)

In [ ]:
df['scores'] = model.decision_function(df[['salary']])
df['anomaly'] = model.predict(df[['salary']])
df.head(10)

,salary,scores,anomaly
0,96454,0.492574,1
1,29557,0.466587,1
2,47175,0.399883,1
3,12419,0.522965,1
4,33333333,-42737.796378,-1
5,90938,0.461231,1
6,76404,0.491208,1
7,85815,0.451803,1
8,9999999,-45681.603049,-1
9,73611,0.495589,1


In [ ]:
#Elliptical Envelope
from sklearn.covariance import EllipticEnvelope
model = EllipticEnvelope(0.01)
model.fit(df[['salary']])

EllipticEnvelope(assume_centered=False, contamination=0.1, random_state=None,
                 store_precision=0.01, support_fraction=None)

In [ ]:
df['scores'] = model.decision_function(df[['salary']])
df['anomaly'] = model.predict(df[['salary']])
df.head(10)

,salary,scores,anomaly
0,96454,-9.894581e-02,-1
1,29557,1.514528e+00,1
2,47175,2.378141e+00,1
3,12419,-2.095885e-01,-1
4,33333333,-1.643630e+06,-1
5,90938,5.365942e-01,1
6,76404,1.778674e+00,1
7,85815,1.045962e+00,1
8,9999999,-1.467882e+05,-1
9,73611,1.945540e+00,1


In [ ]:
#Elliptical Envelope
from sklearn.svm import OneClassSVM
model = OneClassSVM(nu=0.01)
model.fit(df[['salary']])

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='rbf',
            max_iter=-1, nu=0.01, shrinking=True, tol=0.001, verbose=False)

In [ ]:
df['scores'] = model.decision_function(df[['salary']])
df['anomaly'] = model.predict(df[['salary']])
df.head(10)

,salary,scores,anomaly
0,96454,0.029090,1
1,29557,0.008111,1
2,47175,0.014739,1
3,12419,0.000907,1
4,33333333,-29.085615,-1
5,90938,0.027790,1
6,76404,0.023996,1
7,85815,0.026514,1
8,9999999,-27.751652,-1
9,73611,0.023205,1


In [ ]:
df.loc[df['anomaly']==-1].count()

salary     598
scores     598
anomaly    598
dtype: int64

In [ ]:
pip install dtale

     |████████████████████████████████| 7.6MB 2.7MB/s 
     |████████████████████████████████| 1.8MB 36.8MB/s 
     |████████████████████████████████| 194kB 43.8MB/s 
     |████████████████████████████████| 645kB 48.9MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 1.0MB 49.0MB/s 
     |████████████████████████████████| 3.4MB 44.3MB/s 
     |████████████████████████████████| 194kB 38.9MB/s 
     |████████████████████████████████| 1.8MB 48.3MB/s 
     |████████████████████████████████| 358kB 46.9MB/s 
  Created wheel for dash-daq: filename=dash_daq-0.5.0-cp36-none-any.whl size=669704 sha256=22955ab999a931d6ad57645be3ad1076ffad5fc4648eb22f851940cd2e89c4fa
  Stored in directory: /root/.cache/pip/wheels/1c/9c/f7/84731716eaa9fc952cdbdd1f87df3ca9b8805c317c609efcd6
  Created wheel for dash-colorscales: filename=dash_colorscales-0.0.4-cp36-none-any.whl size=62599 sha256=c7b9d0ed8dc766530486

In [ ]:
d = sns.load_dataset('planets')
d.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [ ]:
import dtale 
import dtale.app as dtale_app
dtale_app.USE_COLAB = True

In [ ]:
dtale.show(d)

https://x8zgr5k812l-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/2

In [ ]:
#Import your case study dataset to the dtale package and do an in detail extensive eda of it.